In [11]:
# with open("example_md_to_text.txt", "r", encoding="latin-1") as f: # TO DO: check proper encoding for .md files
#     markdown_example = f.read()

# with open("example.pdf", "rb") as f:
#     pdf = f.read()

# with open("HF_TOKEN.txt", "r") as f:
#     hf_token = f.read()

# with open("GROQ_KEY.txt", "r") as f:
#     groq_token = f.read()

Information about distance metrics supported: https://github.com/pgvector/pgvector


In [12]:
# !pip install psycopg2 pgvector

In [1]:
# # Install postgresql server
# !sudo apt-get -y -qq update
# !sudo apt-get -y -qq install postgresql
# !sudo service postgresql start

# # Setup a password `postgres` for username `postgres`
# !sudo -u postgres psql -U postgres -c "ALTER USER postgres PASSWORD 'postgres';"

# # Setup a database with name `rag` to be used
# !sudo -u postgres psql -U postgres -c 'DROP DATABASE IF EXISTS rag;'
# !sudo -u postgres psql -U postgres -c 'CREATE DATABASE rag;'

Sudo is disabled on this machine. To enable it, go to the ]8;;ms-settings:developers\Developer Settings page]8;;\ in the Settings app
Sudo is disabled on this machine. To enable it, go to the ]8;;ms-settings:developers\Developer Settings page]8;;\ in the Settings app
Sudo is disabled on this machine. To enable it, go to the ]8;;ms-settings:developers\Developer Settings page]8;;\ in the Settings app
error: unexpected argument '-u' found

  tip: to pass '-u' as a value, use '-- -u'

Usage: sudo [OPTIONS] [COMMANDLINE]... [COMMAND]

For more information, try 'help'.
error: unexpected argument '-u' found

  tip: to pass '-u' as a value, use '-- -u'

Usage: sudo [OPTIONS] [COMMANDLINE]... [COMMAND]

For more information, try 'help'.
error: unexpected argument '-u' found

  tip: to pass '-u' as a value, use '-- -u'

Usage: sudo [OPTIONS] [COMMANDLINE]... [COMMAND]

For more information, try 'help'.


In [14]:
!cd /tmp
!git clone --branch v0.8.0 https://github.com/pgvector/pgvector.git
!cd pgvector
!make
!make install # may need sudo

fatal: destination path 'pgvector' already exists and is not an empty directory.
make: *** No targets specified and no makefile found.  Stop.
make: *** No rule to make target 'install'.  Stop.


In [15]:
%env RAG_DATABASE_NAME=rag
%env RAG_DATABASE_HOST=localhost
%env RAG_DATABASE_PORT=5432
%env RAG_DATABASE_USER=postgres
%env RAG_DATABASE_PASS=postgres

env: RAG_DATABASE_NAME=rag
env: RAG_DATABASE_HOST=localhost
env: RAG_DATABASE_PORT=5432
env: RAG_DATABASE_USER=postgres
env: RAG_DATABASE_PASS=postgres


In [16]:
import psycopg2

conn = psycopg2.connect(
    user="postgres",
    password="postgres",
    host="localhost",
    port="5432",
    database="rag",
)
cur = conn.cursor()


In [31]:
statement_enable_pg_vect = "CREATE EXTENSION IF NOT EXISTS vector"

In [32]:
statement_create_initial_table = """CREATE TABLE doc_info (
    doc_id          varchar(80),   -- string of var len with up to 80 characters
    n_pages         int,           -- number of pages in the doc
    n_tokens        int,           -- number of tokens in the doc
    full_text       text,          -- unlimited text dtype, contains all the text
)"""

In [33]:
statement_create_chunks_table = """CREATE TABLE doc_chunks (
    doc_id          varchar(80),   -- string of var len with up to 80 characters
    chunk_id        varchar(80),   -- string of var len with up to 80 characters
    chunk_text      text,          -- text of the chunk
    chunk_embedding vector(768),          -- I need to get the pg vector thing here
    chunk_type      varchar(80),   -- is the chunk a sentence, a paragraph, a summary
    page_ns         vector(768),           -- page number(s) where the chunk appears
    has_table       boolean,       -- true if the chunk contains a table
)"""

In [34]:
statement_create_tables_table = """CREATE TABLE doc_tables (
    doc_id                varchar(80),   -- string of var len with up to 80 characters
    chunk_id              varchar(80),   -- string of var len with up to 80 characters
    table_id              varchar(80),   -- string of var len with up to 80 characters
    page_n                int,           -- page number(s) where the table appears
    markdown_text         text,          -- text of the table in markdown format
    html_text             text,          -- text of the table in htm format
    generated_description text,          -- generated description of the table
    table_descr_embedding vector(768),          -- I need to get the pg vector thing here
    generation_model_id   varchar(80)    -- id of the model used to generate the description
)"""

In [35]:
statement_create_model_id_table = """CREATE TABLE doc_tables (
    generation_model_id   varchar(80)    -- id of the model used to generate the description
    generation_model_name varchar(80),   -- name of the model correspoding to the id
)"""

In [36]:
from pgvector.asyncpg import register_vector

cur.execute(statement_enable_pg_vect)
register_vector(cur)

InFailedSqlTransaction: current transaction is aborted, commands ignored until end of transaction block


In [38]:
cur.execute(statement_create_initial_table)
cur.execute(statement_create_chunks_table)
cur.execute(statement_create_tables_table)
cur.execute(statement_create_model_id_table)

SyntaxError: syntax error at or near ")"
LINE 6: )
        ^
